In [ ]:
import sys as sys
import unyt
import h5py
import logging
import numpy as np
import matplotlib.pyplot as plt
logging.basicConfig(level = logging.INFO)
sys.path.append('/home/jovyan/Analysis/transfer')
sys.path.append('/home/jovyan/Analysis/Pylians3/library/build/lib.linux-x86_64-3.7')
import readgadget
from matplotlib.gridspec import GridSpec 
import matplotlib as mpl
import time
from scipy.spatial import cKDTree
from transfer import LOGGER
import os
import readgadget
from matplotlib.gridspec import GridSpec 
import matplotlib as mpl
import time
import pandas as pd
from scipy.spatial import cKDTree
import statistics as stats
import math
import csv
from scipy import stats
from transfer import LOGGER

#For loading 1P simulations with new naming convention
list_of_strings2 = ["1P_p1_n2", "1P_p1_n1", "1P_p1_0", "1P_p1_1", "1P_p1_2", 
                    "1P_p2_n2", "1P_p2_n1", "1P_p2_0", "1P_p2_1", "1P_p2_2", 
                    "1P_p3_n2", "1P_p3_n1", "1P_p3_0", "1P_p3_1", "1P_p3_2", 
                    "1P_p4_n2", "1P_p4_n1", "1P_p4_0", "1P_p4_1", "1P_p4_2", 
                    "1P_p5_n2", "1P_p5_n1", "1P_p5_0", "1P_p5_1", "1P_p5_2", 
                    "1P_p6_n2", "1P_p6_n1", "1P_p6_0", "1P_p6_1", "1P_p6_2", ]


In [ ]:
#This function selects the particles within the specified halo
def profile_data_selector_radial_fix(halo_index, nbody_index):
    
    CoM = haloes['Group/GroupPos'][halo_index,:]
    radius = nbody_haloes['Group/Group_R_Crit200'][nbody_index]
    
    radius_array = 10**np.arange(-2, 1+dr, dr) * radius
    dark_matter_mask = dark_matter_tree.query_ball_point(x=CoM, r=radialX*radius)
    gas_mask = gas_tree.query_ball_point(x=CoM, r=radialX*radius)
    star_mask = star_tree.query_ball_point(x=CoM, r=radialX*radius)
    bh_mask = bh_tree.query_ball_point(x=CoM, r=radialX*radius)

    #LOGGER.info("Selecting particles in halo")

    dark_matter_in_halo_coords = dark_matter_final_coordinates[dark_matter_mask]
    gas_in_halo_coords = gas_final_coordinates[gas_mask]
    gas_in_halo_masses = gas_final_masses[gas_mask]
    star_in_halo_coords = star_final_coordinates[star_mask]
    star_in_halo_masses = star_final_masses[star_mask]
    bh_in_halo_coords = bh_final_coordinates[bh_mask]
    bh_in_halo_masses = bh_final_masses[bh_mask]

    dm_dx = CoM[0] - dark_matter_in_halo_coords[:,0]
    dm_dy = CoM[1] - dark_matter_in_halo_coords[:,1]
    dm_dz = CoM[2] - dark_matter_in_halo_coords[:,2]

    dark_matter_radial_value = np.sqrt((dm_dx[:]**2) + (dm_dy[:]**2) + (dm_dz[:]**2))
    dark_matter_in_halo_masses = np.zeros(len(dark_matter_radial_value)) + dark_matter_final_masses[0]

    gas_dx = CoM[0] - gas_in_halo_coords[:,0]
    gas_dy = CoM[1] - gas_in_halo_coords[:,1]
    gas_dz = CoM[2] - gas_in_halo_coords[:,2]

    gas_radial_value = np.sqrt((gas_dx[:]**2) + (gas_dy[:]**2) + (gas_dz[:]**2))

    star_dx = CoM[0] - star_in_halo_coords[:,0]
    star_dy = CoM[1] - star_in_halo_coords[:,1]
    star_dz = CoM[2] - star_in_halo_coords[:,2]

    star_radial_value = np.sqrt((star_dx[:]**2) + (star_dy[:]**2) + (star_dz[:]**2))

    bh_dx = CoM[0] - bh_in_halo_coords[:,0]
    bh_dy = CoM[1] - bh_in_halo_coords[:,1]
    bh_dz = CoM[2] - bh_in_halo_coords[:,2]

    bh_radial_value = np.sqrt((bh_dx[:]**2) + (bh_dy[:]**2) + (bh_dz[:]**2))
    

    return dark_matter_in_halo_masses, dark_matter_radial_value, gas_in_halo_masses, gas_radial_value, star_in_halo_masses, star_radial_value, bh_in_halo_masses, bh_radial_value, radius_array, radius

#This function calculates the density profile for a specified halo for each mass component
def total_profile_calculator_radial_fix(bins, dm_InHaloMasses, gas_InHaloMasses, star_InHaloMasses, 
                                        bh_InHaloMasses, dark_matter_radial_value, gas_radial_value, 
                                        star_radial_value, bh_radial_value, radius_array, radius):
    
    dm_mass_array = np.zeros(bins)
    gas_mass_array = np.zeros(bins)
    star_mass_array = np.zeros(bins)
    bh_mass_array = np.zeros(bins)
    total_mass_array = np.zeros(bins)
    
    total_density_array = np.zeros(bins)
    dm_density_array = np.zeros(bins)
    gas_density_array = np.zeros(bins)
    star_density_array = np.zeros(bins)
    bh_density_array = np.zeros(bins)
    
    volume_prev = 0
    prev_radius = 0
    for j in range(bins):
        dm_IN_mask = (dark_matter_radial_value < radius_array[j]) & (dark_matter_radial_value > prev_radius)
        gas_IN_mask = (gas_radial_value < radius_array[j]) & (gas_radial_value > prev_radius)
        star_IN_mask = (star_radial_value < radius_array[j]) & (star_radial_value > prev_radius)
        bh_IN_mask = (bh_radial_value < radius_array[j]) & (bh_radial_value > prev_radius)

        dm_enclosed_mass = dm_InHaloMasses[dm_IN_mask]
        gas_enclosed_mass = gas_InHaloMasses[gas_IN_mask]
        star_enclosed_mass = star_InHaloMasses[star_IN_mask]
        bh_enclosed_mass = bh_InHaloMasses[bh_IN_mask]

        dm_mass_array[j] = np.sum(dm_enclosed_mass)
        gas_mass_array[j] = np.sum(gas_enclosed_mass)
        star_mass_array[j] = np.sum(star_enclosed_mass)
        bh_mass_array[j] = np.sum(bh_enclosed_mass)
        total_mass_array[j] = dm_mass_array[j]+gas_mass_array[j]+star_mass_array[j]+bh_mass_array[j]

        volume = (4/3)*(np.pi)*((radius_array[j])**3)
        
        total_density_array[j] = (total_mass_array[j])/(volume-volume_prev)
        dm_density_array[j] = (dm_mass_array[j])/(volume-volume_prev)
        gas_density_array[j] = (gas_mass_array[j])/(volume-volume_prev)
        star_density_array[j] = (star_mass_array[j])/(volume-volume_prev)
        bh_density_array[j] = (bh_mass_array[j])/(volume-volume_prev)
        prev_radius = radius_array[j]
        volume_prev = (4/3)*(np.pi)*((radius_array[j])**3)


    return total_density_array, dm_density_array, gas_density_array, star_density_array, bh_density_array

#This function calculates the mass profile for a specified halo for each mass component
def total_mass_profile_calculator_radial_fix(bins, dm_InHaloMasses, gas_InHaloMasses, star_InHaloMasses, 
                                        bh_InHaloMasses, dark_matter_radial_value, gas_radial_value, 
                                        star_radial_value, bh_radial_value, radius_array, radius):
    
    dm_mass_array = np.zeros(bins)
    gas_mass_array = np.zeros(bins)
    star_mass_array = np.zeros(bins)
    bh_mass_array = np.zeros(bins)
    total_mass_array = np.zeros(bins)
    
    total_density_array = np.zeros(bins)
    dm_density_array = np.zeros(bins)
    gas_density_array = np.zeros(bins)
    star_density_array = np.zeros(bins)
    bh_density_array = np.zeros(bins)
    
    integrated_total_mass_array = np.zeros(bins)
    integrated_dm_mass_array = np.zeros(bins)
    integrated_gas_mass_array = np.zeros(bins)
    integrated_star_mass_array = np.zeros(bins)
    integrated_bh_mass_array = np.zeros(bins)
    
    volume_prev = 0
    prev_radius = 0
    for j in range(bins):
        dm_IN_mask = (dark_matter_radial_value < radius_array[j]) & (dark_matter_radial_value > prev_radius)
        gas_IN_mask = (gas_radial_value < radius_array[j]) & (gas_radial_value > prev_radius)
        star_IN_mask = (star_radial_value < radius_array[j]) & (star_radial_value > prev_radius)
        bh_IN_mask = (bh_radial_value < radius_array[j]) & (bh_radial_value > prev_radius)

        dm_enclosed_mass = dm_InHaloMasses[dm_IN_mask]
        gas_enclosed_mass = gas_InHaloMasses[gas_IN_mask]
        star_enclosed_mass = star_InHaloMasses[star_IN_mask]
        bh_enclosed_mass = bh_InHaloMasses[bh_IN_mask]

        dm_mass_array[j] = np.sum(dm_enclosed_mass)
        gas_mass_array[j] = np.sum(gas_enclosed_mass)
        star_mass_array[j] = np.sum(star_enclosed_mass)
        bh_mass_array[j] = np.sum(bh_enclosed_mass)
        total_mass_array[j] = dm_mass_array[j]+gas_mass_array[j]+star_mass_array[j]+bh_mass_array[j]

        integrated_total_mass_array[j] = np.sum(total_mass_array)
        integrated_dm_mass_array[j] = np.sum(dm_mass_array)
        integrated_gas_mass_array[j] = np.sum(gas_mass_array)
        integrated_star_mass_array[j] = np.sum(star_mass_array)
        integrated_bh_mass_array[j] = np.sum(bh_mass_array)
        prev_radius = radius_array[j]



    return integrated_total_mass_array, integrated_dm_mass_array, integrated_gas_mass_array, integrated_star_mass_array, integrated_bh_mass_array

#This function selects the particles within the specified N-body halo
def nbody_profile_data_selector(halo_index):
    CoM = nbody_haloes['Group/GroupPos'][halo_index,:]
    radius = nbody_haloes['Group/Group_R_Crit200'][halo_index]
    
    radius_array = 10**np.arange(-2, 1+dr, dr) * radius
    nbody_dark_matter_mask = nbody_dark_matter_tree.query_ball_point(x=CoM, r=radialX*radius)

    dark_matter_in_halo_coords = nbody_dark_matter_final_coordinates[nbody_dark_matter_mask]
    dark_matter_in_halo_masses = nbody_dark_matter_final_masses[nbody_dark_matter_mask]

    dm_dx = CoM[0] - dark_matter_in_halo_coords[:,0]
    dm_dy = CoM[1] - dark_matter_in_halo_coords[:,1]
    dm_dz = CoM[2] - dark_matter_in_halo_coords[:,2]

    dark_matter_radial_value = np.sqrt((dm_dx[:]**2) + (dm_dy[:]**2) + (dm_dz[:]**2))

    return dark_matter_in_halo_masses, dark_matter_radial_value, radius_array, radius

#This function calculates the density profile for a specified N-body halo for each mass component
def nbody_profile_calculator(bins, InHaloMasses, radial_value, radius_array, radius):
    density_array = np.zeros(bins)
    mass_array = np.zeros(bins)
    volume_prev = 0
    prev_radius = 0
    for k in range(bins):
        enclosed_mass = InHaloMasses[(radial_value < radius_array[k]) & (radial_value > prev_radius)]
        mass_array[k] = np.sum(enclosed_mass)
        volume = (4/3)*(np.pi)*((radius_array[k])**3)
        density_array[k] = (mass_array[k])/(volume-volume_prev)
        prev_radius = radius_array[k]
        volume_prev = (4/3)*(np.pi)*((radius_array[k])**3)
        
    return density_array

#This function calculates the mass profile for a specified N-body halo for each mass component
def nbody_mass_profile_calculator(bins, InHaloMasses, radial_value, radius_array, radius):
    mass_array = np.zeros(bins)
    integrated_mass_array = np.zeros(bins)
    prev_radius = 0
    for k in range(bins):
        enclosed_mass = InHaloMasses[(radial_value < radius_array[k]) & (radial_value > prev_radius)]
        mass_array[k] = np.sum(enclosed_mass)
        integrated_mass_array[k] = np.sum(mass_array)
        prev_radius = radius_array[k]
    
    return integrated_mass_array

#This function calculates the virial mass for a specified halo
def virial_mass_calculator(halo_index):
    
    CoM = haloes['Group/GroupPos'][halo_index,:]
    radius = haloes['Group/Group_R_Crit200'][halo_index]
    
    dark_matter_mask = dark_matter_tree.query_ball_point(x=CoM, r=radius)
    gas_mask = gas_tree.query_ball_point(x=CoM, r=radius)
    star_mask = star_tree.query_ball_point(x=CoM, r=radius)
    bh_mask = bh_tree.query_ball_point(x=CoM, r=radius)

    dark_matter_in_halo_masses = dark_matter_final_masses[dark_matter_mask]
    gas_in_halo_masses = gas_final_masses[gas_mask]
    star_in_halo_masses = star_final_masses[star_mask]
    bh_in_halo_masses = bh_final_masses[bh_mask]
    
    dark_matter_virial_mass = np.sum(dark_matter_in_halo_masses)
    gas_virial_mass = np.sum(gas_in_halo_masses)
    star_virial_mass = np.sum(star_in_halo_masses)
    bh_virial_mass = np.sum(bh_in_halo_masses)
    
    total_virial_mass = dark_matter_virial_mass + gas_virial_mass + star_virial_mass + bh_virial_mass
    
    return total_virial_mass, dark_matter_virial_mass, gas_virial_mass, star_virial_mass, bh_virial_mass, radius

#This function calculates the virial mass for a specified halo
def nbody_virial_mass_calculator(halo_index):
    
    CoM = nbody_haloes['Group/GroupPos'][halo_index,:]
    radius = nbody_haloes['Group/Group_R_Crit200'][halo_index]
    
    dark_matter_mask = nbody_dark_matter_tree.query_ball_point(x=CoM, r=radius)

    dark_matter_in_halo_masses = nbody_dark_matter_final_masses[dark_matter_mask]
    
    dark_matter_virial_mass = np.sum(dark_matter_in_halo_masses)
    
    return dark_matter_virial_mass, radius

#This function matches haloes across two specified simulations
'''
Arguments:
target_simulation: name of simulation to match to N-body (e.g. "CV_0")
target_suite: name of simulation suite, (e.g. "SIMBA")
starting_simulation: name of simulation to be matched instead of N-body (e.g. "CV_0")
starting_suite: name of simulation suite to be matched instead of N-body (e.g. "SIMBA")
run: simulation number (relevant only for 1P simulations, e.g. the 0 in "CV_0")
two_hydro: True if we are matching to another hydro sim, False if matching to N-body

'''
def halo_matcher(target_simulation, target_suite,  starting_simulation = "nothing", starting_suite = "nothing", 
                 run=0, two_hydro = False, snap_num = "090"):
    
    start = time.time()

    #snap_num = "044"
    astrid_snap_num = snap_num
    
    if two_hydro == False:
        if target_simulation[:2] == "1P":
            print("1P simulation")
            if target_suite == "SIMBA":
                if run < 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/{target_simulation[:2]}/' + target_simulation
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/{target_simulation[:2]}/' + target_simulation
                if run >= 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/{target_simulation[:2]}/1P_p1_0'
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/{target_simulation[:2]}/1P_p1_0'
                halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA/{target_simulation[:2]}/' + target_simulation
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA/{target_simulation[:2]}/' + target_simulation
                starting_final_filename = starting_simdir + f'/snapshot_{snap_num}.hdf5'
                final_filename = simdir + f'/snapshot_{snap_num}.hdf5'
                starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'

            if target_suite == "IllustrisTNG":
                if run < 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/IllustrisTNG_DM/{target_simulation[:2]}/' + target_simulation
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/IllustrisTNG_DM/{target_simulation[:2]}/' + target_simulation
                if run >= 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/IllustrisTNG_DM/{target_simulation[:2]}/1P_p1_0'
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/IllustrisTNG_DM/{target_simulation[:2]}/1P_p1_0'
                halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/IllustrisTNG/{target_simulation[:2]}/' + target_simulation
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/IllustrisTNG/{target_simulation[:2]}/' + target_simulation
                starting_final_filename = starting_simdir + f'/snapshot_{snap_num}.hdf5'
                final_filename = simdir + f'/snapshot_{snap_num}.hdf5'
                starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'
                
                #if run < 22:
                #    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/IllustrisTNG_DM/{target_simulation[:2]}/' + target_simulation
                #    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/IllustrisTNG_DM/{target_simulation[:2]}/' + target_simulation
                #if run >= 22:
                #    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/IllustrisTNG_DM/{target_simulation[:2]}/1P_p1_0'
                #    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/IllustrisTNG_DM/{target_simulation[:2]}/1P_p1_0'
                #halodir = f'/home/jovyan/FOF_Subfind/IllustrisTNG_extended_1P/1P/' + target_simulation
                #simdir = f'/home/jovyan/Sims/IllustrisTNG_extended_1P/1P/' + target_simulation
                #starting_final_filename = starting_simdir + f'/snapshot_{snap_num}.hdf5'
                #final_filename = simdir + f'/snapshot_{snap_num}.hdf5'
                #starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                #final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'

            if target_suite == "Astrid":
                if run < 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/Astrid_DM/1P/' + target_simulation
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/Astrid_DM/1P/' + target_simulation
                if run >= 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/Astrid_DM/1P/1P_p1_0'
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/Astrid_DM/1P/1P_p1_0'
                halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/Astrid/1P/' + target_simulation
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/Astrid/1P/' + target_simulation
                starting_final_filename = starting_simdir + f'/snapshot_{astrid_snap_num}.hdf5'
                final_filename = simdir + f'/snapshot_{astrid_snap_num}.hdf5'
                starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'
                
            if target_suite == "Swift-EAGLE":
                if run < 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/{target_simulation[:2]}/' + target_simulation
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/{target_simulation[:2]}/' + target_simulation
                if run >= 10:
                    starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/{target_simulation[:2]}/1P_1_0'
                    starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/{target_simulation[:2]}/1P_1_0'
                halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/Swift-EAGLE/L25n256/{target_simulation[:2]}/' + target_simulation
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/Swift-EAGLE/L25n256/{target_simulation[:2]}/' + target_simulation
                starting_final_filename = starting_simdir + f'/snapshot_{snap_num}.hdf5'
                final_filename = simdir + f'/snapshot_{snap_num}.hdf5'
                starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'
                
                
        else:
            if target_suite == "SIMBA":
                starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/{target_simulation[:2]}/' + target_simulation
                starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/{target_simulation[:2]}/' + target_simulation
                halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA/{target_simulation[:2]}/' + target_simulation
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA/{target_simulation[:2]}/' + target_simulation

                starting_final_filename = starting_simdir + f'/snapshot_{snap_num}.hdf5'
                final_filename = simdir + f'/snapshot_{snap_num}.hdf5'
                starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'

            if target_suite == "IllustrisTNG":
                starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/IllustrisTNG_DM/{target_simulation[:2]}/' + target_simulation
                starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/IllustrisTNG_DM/{target_simulation[:2]}/' + target_simulation
                halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/IllustrisTNG/{target_simulation[:2]}/' + target_simulation
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/IllustrisTNG/{target_simulation[:2]}/' + target_simulation

                starting_final_filename = starting_simdir + f'/snapshot_{snap_num}.hdf5'
                final_filename = simdir + f'/snapshot_{snap_num}.hdf5'
                starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'

            if target_suite == "Astrid":
                starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/Astrid_DM/{target_simulation[:3]}/' + target_simulation
                starting_final_filename = starting_simdir + f'/snapshot_{astrid_snap_num}.hdf5'
                starting_final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/Astrid_DM/{target_simulation[:3]}/{target_simulation}' + f'/fof_subhalo_tab_{astrid_snap_num}.hdf5'

                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/Astrid/{target_simulation[:3]}/' + target_simulation
                final_filename = simdir + f'/snapshot_{astrid_snap_num}.hdf5'
                final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/Astrid/{target_simulation[:3]}/{target_simulation}' + f'/groups_{snap_num}.hdf5'

            if target_suite == "Swift-EAGLE":
                starting_halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/L25n256/{target_simulation[:2]}/' + target_simulation
                starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/L25n256/{target_simulation[:2]}/' + target_simulation
                halodir = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/Swift-EAGLE/L25n256/{target_simulation[:2]}/' + target_simulation
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/Swift-EAGLE/L25n256/{target_simulation[:2]}/' + target_simulation

                starting_final_filename = starting_simdir + f'/snapshot_{snap_num}.hdf5'
                final_filename = simdir + f'/snapshot_{snap_num}.hdf5'
                starting_final_halo_filename = starting_halodir + f'/groups_{snap_num}.hdf5'
                final_halo_filename = halodir + f'/groups_{snap_num}.hdf5'
                
                
    if two_hydro == True:
        LOGGER.info("Running in double-hydro mode")
        if starting_suite == "SIMBA":
            #starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA/{starting_simulation[:2]}/' + starting_simulation
            #starting_final_filename = starting_simdir + f'/snap_{snap_num}.hdf5'
            #starting_final_halo_filename = starting_simdir + f'/fof_subhalo_tab_{snap_num}.hdf5'
            
            starting_simdir = f'/home/jovyan/Sims/SIMBA_extended_1P/' + starting_simulation
            starting_final_filename = starting_simdir + f'/snap_{snap_num}.hdf5'
            starting_final_halo_filename = f'/home/jovyan/FOF_Subfind/SIMBA_extended_1P/{starting_simulation}/' + f'fof_subhalo_tab_{snap_num}.hdf5'

        if starting_suite == "IllustrisTNG":
            #starting_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{target_suite}/{starting_simulation[:2]}/' + starting_simulation
            #starting_final_filename = starting_simdir + f'/snap_{snap_num}.hdf5'
            #starting_final_halo_filename = starting_simdir + f'/fof_subhalo_tab_{snap_num}.hdf5'
            
            starting_simdir = f'/home/jovyan/Sims/IllustrisTNG_extended_1P/1P/' + starting_simulation
            starting_final_filename = starting_simdir + f'/snap_{snap_num}.hdf5'
            starting_final_halo_filename = f'/home/jovyan/FOF_Subfind/IllustrisTNG_extended_1P/1P/{starting_simulation}/' + f'fof_subhalo_tab_{snap_num}.hdf5'

        if starting_suite == "Astrid":
            starting_simdir = f'/home/jovyan/Sims/{target_suite}/{starting_simulation[:2]}/' + starting_simulation
            starting_final_filename = starting_simdir + f'/snap_{astrid_snap_num}.hdf5'
            starting_final_halo_filename = f'/home/jovyan/FOF_Subfind/Astrid/{starting_simulation[:2]}/{starting_simulation}' + f'/fof_subhalo_tab_{astrid_snap_num}.hdf5'
            
        if target_suite == "SIMBA":
            simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{target_suite}/{target_simulation[:2]}/' + target_simulation
            final_filename = simdir + f'/snap_{snap_num}.hdf5'
            final_halo_filename = simdir + f'/fof_subhalo_tab_{snap_num}.hdf5'

        if target_suite == "IllustrisTNG":
            simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{target_suite}/{target_simulation[:2]}/' + target_simulation
            final_filename = simdir + f'/snap_{snap_num}.hdf5'
            final_halo_filename = simdir + f'/fof_subhalo_tab_{snap_num}.hdf5'

        if target_suite == "Astrid":
            simdir = f'/home/jovyan/Sims/{target_suite}/{target_simulation[:2]}/' + target_simulation
            final_filename = simdir + f'/snap_{astrid_snap_num}.hdf5'
            final_halo_filename = simdir + f'/fof_subhalo_tab_{astrid_snap_num}.hdf5'
            
    ############################################################################################################################
    LOGGER.info("Loading N-body data")
    ###########################################################################################################################
    if two_hydro == False:
        with h5py.File(starting_final_filename, "r") as starting_handle:
            starting_dark_matter_final_coordinates = np.array(
                starting_handle["PartType1/Coordinates"][:]
            )
            starting_dark_matter_final_ids = np.array(
                starting_handle["PartType1/ParticleIDs"][:]
            )

        starting_haloes = h5py.File(starting_final_halo_filename, 'r')
        starting_FOF_ID_array = starting_haloes['IDs/ID'][:]
        
    if two_hydro == True:
        LOGGER.info("Loading double-hydro data")
        with h5py.File(starting_final_filename, "r") as starting_handle:
            starting_dark_matter_final_coordinates = np.array(
                starting_handle["PartType1/Coordinates"][:]
            )
            starting_dark_matter_final_ids = np.array(
                starting_handle["PartType1/ParticleIDs"][:]
            )

        starting_haloes = h5py.File(starting_final_halo_filename, 'r')
        #starting_FOF_ID_array = starting_haloes['IDs/ID'][:]
        
        if (starting_suite == "SIMBA") or (starting_suite == "Astrid") or (starting_suite == "Swift-EAGLE"):
            starting_FOF_ID_array = starting_haloes['IDs/ID'][:]
            starting_only_dm_FOF_ID_array = starting_FOF_ID_array[np.isin(starting_FOF_ID_array, starting_dark_matter_final_ids)]
            out_halo_mask = np.where(np.isin(starting_dark_matter_final_ids, starting_only_dm_FOF_ID_array) == False)[0]
            out_halo_ids = starting_dark_matter_final_ids[out_halo_mask]
            starting_total_FOF_ID_array = np.append(starting_only_dm_FOF_ID_array, out_halo_ids)
            num_parts = len(starting_only_dm_FOF_ID_array)

        if starting_suite == "IllustrisTNG":
            starting_total_FOF_ID_array = starting_dark_matter_final_ids
            num_parts = len(starting_dark_matter_final_coordinates)
            
    ############################################################################################################################
    LOGGER.info("Loading Hydro data")
    ############################################################################################################################

    with h5py.File(final_filename, "r") as handle:
        dark_matter_final_coordinates = np.array(
            handle["PartType1/Coordinates"][:]
        )
        dark_matter_final_ids = np.array(
            handle["PartType1/ParticleIDs"][:]
        )

    haloes = h5py.File(final_halo_filename, 'r')

    num_particles_in_hydro = starting_haloes['Group/GroupLen'][:]
    mass_mask = (num_particles_in_hydro > 100)
    halo_mass_limit = np.where(mass_mask == True)[0]
    group_len_type_all = np.array(haloes['Group/GroupLenType'][:,1])

    if (target_suite == "SIMBA") or (target_suite == "Astrid") or (target_suite == "Swift-EAGLE"):
        FOF_ID_array = haloes['IDs/ID'][:]
        only_dm_FOF_ID_array = FOF_ID_array[np.isin(FOF_ID_array, dark_matter_final_ids)]
        out_halo_mask = np.where(np.isin(dark_matter_final_ids, only_dm_FOF_ID_array) == False)[0]
        out_halo_ids = dark_matter_final_ids[out_halo_mask]
        total_FOF_ID_array = np.append(only_dm_FOF_ID_array, out_halo_ids)
        num_parts = len(only_dm_FOF_ID_array)

    if target_suite == "IllustrisTNG":
        total_FOF_ID_array = dark_matter_final_ids
        num_parts = len(dark_matter_final_ids)

    if target_suite == "Swift-EAGLE":
        dark_matter_final_coordinates = dark_matter_final_coordinates * 0.6711 * 1000
        
        with h5py.File(starting_final_filename, "r") as handle:
            nbody_dark_matter_final_coordinates = np.array(
                handle["PartType1/Coordinates"][:]
            )
            nbody_dark_matter_final_ids = np.array(
                handle["PartType1/ParticleIDs"][:]
            )
        
        if (target_simulation == "CV_2") or (target_simulation == "CV_8") or (target_simulation == "CV_17"):
            starting_initial_filename = f"/home/jovyan/home/fixed_SIMBA_DM/{target_simulation}/ICs/ics"
        else:      
            starting_initial_filename = starting_simdir + "/ICs/ics"
        initial_filename = simdir + '/ICs/ics.hdf5'
        
        with h5py.File(initial_filename, "r") as handle:
            dark_matter_initial_ids = np.sort(handle["PartType1/ParticleIDs"][:]) * 2
            dark_matter_initial_coordinates = handle['PartType1/Coordinates'][:] * 0.6711 * 1000
            
        header = readgadget.header(starting_initial_filename)
        nbody_dark_matter_initial_coordinates = readgadget.read_block(starting_initial_filename, "POS ", [1], verbose=True)
        nbody_dark_matter_initial_ids = readgadget.read_block(starting_initial_filename, "ID  ", [1], verbose=True) 
        boxsize = 25000
        
        #checks for negative values
        neg_check_0 = np.where(dark_matter_initial_coordinates[:,0]<0)[0]
        if neg_check_0.any() == True:
            dark_matter_initial_coordinates[neg_check_0, 0] = dark_matter_initial_coordinates[neg_check_0, 0] + boxsize
            LOGGER.info(f"Fixed index {neg_check_0} of the X coordinate")

        neg_check_1 = np.where(dark_matter_initial_coordinates[:,1]<0)[0]
        if neg_check_1.any() == True:
            dark_matter_initial_coordinates[neg_check_1, 1] = dark_matter_initial_coordinates[neg_check_1, 1] + boxsize
            LOGGER.info(f"Fixed index {neg_check_1} of the X coordinate")

        neg_check_2 = np.where(dark_matter_initial_coordinates[:,2]<0)[0]
        if neg_check_2.any() == True:
            dark_matter_initial_coordinates[neg_check_2, 2] = dark_matter_initial_coordinates[neg_check_2, 2] + boxsize
            LOGGER.info(f"Fixed index {neg_check_2} of the X coordinate")

        ind_check0 = np.where(dark_matter_initial_coordinates[:,0] >= boxsize)[0]
        if ind_check0.any() == True:
            dark_matter_initial_coordinates[ind_check0, 0] = boxsize - 0.001
            LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
        ind_check1 = np.where(dark_matter_initial_coordinates[:,1] >= boxsize)[0]
        if ind_check1.any() == True:
            dark_matter_initial_coordinates[ind_check1, 1] = boxsize - 0.001
            LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
        ind_check2 = np.where(dark_matter_initial_coordinates[:,2] >= boxsize)[0]
        if ind_check2.any() == True:
            dark_matter_initial_coordinates[ind_check2, 2] = boxsize - 0.001
            LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

        ind_check0 = np.where(nbody_dark_matter_initial_coordinates[:,0] >= boxsize)[0]
        if ind_check0.any() == True:
            nbody_dark_matter_initial_coordinates[ind_check0, 0] = boxsize - 0.001
            LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
        ind_check1 = np.where(nbody_dark_matter_initial_coordinates[:,1] >= boxsize)[0]
        if ind_check1.any() == True:
            nbody_dark_matter_initial_coordinates[ind_check1, 1] = boxsize - 0.001
            LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
        ind_check2 = np.where(nbody_dark_matter_initial_coordinates[:,2] >= boxsize)[0]
        if ind_check2.any() == True:
            nbody_dark_matter_initial_coordinates[ind_check2, 2] = boxsize - 0.001
            LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate")
            
        tree = cKDTree(dark_matter_initial_coordinates, boxsize=25000)
        distances, initial_hydro_to_nbody = tree.query(x = nbody_dark_matter_initial_coordinates, k=1, workers=-1)
        
        tree2 = cKDTree(nbody_dark_matter_initial_coordinates, boxsize=25000)
        distances, initial_nbody_to_hydro = tree2.query(x = dark_matter_initial_coordinates, k=1, workers=-1)
        
        fixed_nbody_initial = nbody_dark_matter_initial_ids[initial_nbody_to_hydro]
        
        dictionary = pd.Series(fixed_nbody_initial,index=dark_matter_initial_ids).to_dict()
        
        nbody_numbered_hydro_final_ids = np.empty(len(fixed_nbody_initial), dtype = int)
        for i in range(len(fixed_nbody_initial)):
            value = dark_matter_final_ids[i]
            nbody_numbered_hydro_final_ids[i] = int(dictionary[value])
            
        initial_IDs = pd.DataFrame(nbody_numbered_hydro_final_ids, columns = ["IDs"])
        final_IDs = pd.DataFrame(nbody_dark_matter_final_ids, columns = ["IDs"])

        initial_IDs['index'] = initial_IDs.index 
        initial_IDs = initial_IDs.set_index('IDs')
        initial_IDs = initial_IDs.reindex(index=final_IDs['IDs'])
        initial_IDs = initial_IDs.reset_index()

        hydro_to_nbody = initial_IDs['index'].tolist()
        
        initial_IDs = pd.DataFrame(nbody_dark_matter_final_ids, columns = ["IDs"])
        final_IDs = pd.DataFrame(nbody_numbered_hydro_final_ids, columns = ["IDs"])

        initial_IDs['index'] = initial_IDs.index 
        initial_IDs = initial_IDs.set_index('IDs')
        initial_IDs = initial_IDs.reindex(index=final_IDs['IDs'])
        initial_IDs = initial_IDs.reset_index()

        nbody_to_hydro = initial_IDs['index'].tolist()
        
        #initial_IDs = pd.DataFrame(dark_matter_initial_ids, columns = ["IDs"])  ##These dont match, need to do the
        #final_IDs = pd.DataFrame(dark_matter_final_ids, columns = ["IDs"])      

        #initial_IDs['index'] = initial_IDs.index 
        #initial_IDs = initial_IDs.set_index('IDs')
        #initial_IDs = initial_IDs.reindex(index=final_IDs['IDs'])
        #initial_IDs = initial_IDs.reset_index()

        #hydro_initial_to_final = initial_IDs['index'].tolist()
        
        
        #initial_IDs = pd.DataFrame(nbody_dark_matter_initial_ids, columns = ["IDs"])
        #final_IDs = pd.DataFrame(nbody_dark_matter_final_ids, columns = ["IDs"])

        #initial_IDs['index'] = initial_IDs.index 
        #initial_IDs = initial_IDs.set_index('IDs')
        #initial_IDs = initial_IDs.reindex(index=final_IDs['IDs'])
        #initial_IDs = initial_IDs.reset_index()

        #nbody_initial_to_final = initial_IDs['index'].tolist()
        
        #nbody_to_hydro = initial_nbody_to_hydro[hydro_initial_to_final]
        #hydro_to_nbody = initial_hydro_to_nbody[nbody_initial_to_final]
        
        
    ############################################################################################################################
    LOGGER.info("Making Hydro ID array")
    ############################################################################################################################
    num_hydro_haloes = len(halo_mass_limit)
    my_halo_ids = np.zeros(num_parts) - 1
    loop_start_index = 0
    for i in range(num_hydro_haloes):
        next_halo = np.repeat(i, group_len_type_all[i])
        end_index = loop_start_index + group_len_type_all[i]
        my_halo_ids[loop_start_index:end_index] = next_halo
        loop_start_index = loop_start_index + group_len_type_all[i]

    #Attaching out-of-halo particles
    part_diff = len(total_FOF_ID_array) - len(my_halo_ids)
    next_halo = np.repeat(-1, part_diff)
    my_halo_ids = np.append(my_halo_ids, next_halo)

    #ID matching between halo ID array and final dark matter IDs
    hydro_IDs = pd.DataFrame(total_FOF_ID_array, columns = ["IDs"])
    nbody_IDs = pd.DataFrame(dark_matter_final_ids, columns = ["IDs"])

    hydro_IDs['index'] = hydro_IDs.index 
    hydro_IDs = hydro_IDs.set_index('IDs')
    hydro_IDs = hydro_IDs.reindex(index=nbody_IDs['IDs'])
    hydro_IDs = hydro_IDs.reset_index()

    halo_id_indices = hydro_IDs['index'].tolist()

    hydro_id_array = my_halo_ids[halo_id_indices]

    ############################################################################################################################
    LOGGER.info("Making N-body ID array")
    ############################################################################################################################
    if two_hydro == False:
        starting_out_halo_mask = np.where(np.isin(starting_dark_matter_final_ids, starting_FOF_ID_array) == False)[0]
        starting_out_halo_ids = starting_dark_matter_final_ids[starting_out_halo_mask]
        starting_total_FOF_ID_array = np.append(starting_FOF_ID_array, starting_out_halo_ids)
        num_parts = len(starting_FOF_ID_array)
    starting_group_len_type_all = np.array(starting_haloes['Group/GroupLenType'][:,1])
    starting_my_halo_ids = np.array([])

    num_nbody_haloes = len(halo_mass_limit)
    num_parts = len(starting_total_FOF_ID_array)
    starting_my_halo_ids = np.zeros(num_parts) - 1
    starting_loop_start_index = 0
    for i in range(num_nbody_haloes):
        next_halo = np.repeat(i, starting_group_len_type_all[i])
        end_index = starting_loop_start_index + starting_group_len_type_all[i]
        starting_my_halo_ids[starting_loop_start_index:end_index] = next_halo
        starting_loop_start_index = starting_loop_start_index + starting_group_len_type_all[i]

    part_diff = len(starting_total_FOF_ID_array) - len(starting_my_halo_ids)
    next_halo = np.repeat(-1, part_diff)
    starting_my_halo_ids = np.append(starting_my_halo_ids, next_halo)

    hydro_IDs = pd.DataFrame(starting_total_FOF_ID_array, columns = ["IDs"])
    nbody_IDs = pd.DataFrame(starting_dark_matter_final_ids, columns = ["IDs"])

    hydro_IDs['index'] = hydro_IDs.index  
    hydro_IDs = hydro_IDs.set_index('IDs')
    hydro_IDs = hydro_IDs.reindex(index=nbody_IDs['IDs'])
    hydro_IDs = hydro_IDs.reset_index()

    starting_halo_id_indices = hydro_IDs['index'].tolist()

    starting_id_array = starting_my_halo_ids[starting_halo_id_indices]

    ############################################################################################################################
    LOGGER.info("Matching halo ID arrays")
    ############################################################################################################################


    if target_suite == "Astrid":
        LOGGER.info("Doing some extra stuff for Astrid")

        initial_filename = f"/home/jovyan/PUBLIC_RELEASE/Sims/Astrid/{target_simulation[:2]}/{target_simulation}/" + 'ICs/ics' 

        # --- read ICs
        header = readgadget.header(initial_filename)
        dark_matter_initial_coordinates = readgadget.read_block(initial_filename, "POS ", [1], verbose=True)
        OG_ids = readgadget.read_block(initial_filename, "ID  ", [1], verbose=True)
        dark_matter_initial_ids = np.arange(256**3) + 1

        original_final_ids = OG_ids[dark_matter_final_ids - 1]

        sorted_og_final_ids = np.sort(original_final_ids)
        og_final_ids_sort_mask = np.argsort(original_final_ids)
        sorted_halo_ids = hydro_id_array[og_final_ids_sort_mask]

        fixed_hydro = sorted_og_final_ids - 64**3 * ((sorted_og_final_ids/(1+2*64**3)).astype(int) + 1)   


    if (target_suite == "SIMBA") or (target_suite == "IllustrisTNG"):
        sort_mask = np.argsort(dark_matter_final_ids)
        loading_hydro = np.sort(dark_matter_final_ids)
        fixed_hydro = loading_hydro - 64**3 * ((loading_hydro/(1+2*64**3)).astype(int) + 1)   
        sorted_halo_ids = hydro_id_array[sort_mask]
        
    if target_suite == "Swift-EAGLE":
    #    sort_mask = np.argsort(dark_matter_final_ids)
    #    loading_hydro = np.sort(dark_matter_final_ids)
        fixed_hydro = dark_matter_final_ids
        sorted_halo_ids = hydro_id_array

    if two_hydro == False:
        loading_dm = starting_dark_matter_final_ids
    if two_hydro == True:
        if (starting_suite == "SIMBA") or (starting_suite == "IllustrisTNG"):
            starting_sort_mask = np.argsort(starting_dark_matter_final_ids)
            sorted_starting_ids = np.sort(starting_dark_matter_final_ids)
            loading_dm = sorted_starting_ids - 64**3 * ((sorted_starting_ids/(1+2*64**3)).astype(int) + 1)   
            starting_id_array = starting_id_array[starting_sort_mask]
        if starting_suite == "Astrid":
            LOGGER.info("Doing some extra stuff for Astrid")

            initial_filename = f"/home/jovyan/Sims/Astrid/{starting_simulation[:2]}/{starting_simulation}/" + 'ICs/ics'  

            # --- read ICs
            header = readgadget.header(initial_filename)
            dark_matter_initial_coordinates = readgadget.read_block(initial_filename, "POS ", [1], verbose=True)
            OG_ids = readgadget.read_block(initial_filename, "ID  ", [1], verbose=True)
            dark_matter_initial_ids = np.arange(256**3) + 1

            original_final_ids = OG_ids[starting_dark_matter_final_ids - 1]

            sorted_og_final_ids = np.sort(original_final_ids)
            og_final_ids_sort_mask = np.argsort(original_final_ids)
            starting_id_array = starting_id_array[og_final_ids_sort_mask]

            loading_dm = sorted_og_final_ids - 64**3 * ((sorted_og_final_ids/(1+2*64**3)).astype(int) + 1)
    
    #print(max(fixed_hydro))
    #print(max(loading_dm))
    #print(min(fixed_hydro))
    #print(min(loading_dm))
    
    if target_suite != "Swift-EAGLE":
    
        #reordering halo ID arrays to match each other
        hydro_IDs = pd.DataFrame(fixed_hydro, columns = ["IDs"])
        nbody_IDs = pd.DataFrame(loading_dm, columns = ["IDs"])

        hydro_IDs['index'] = hydro_IDs.index 
        hydro_IDs = hydro_IDs.set_index('IDs')
        hydro_IDs = hydro_IDs.reindex(index=nbody_IDs['IDs'])
        hydro_IDs = hydro_IDs.reset_index()

        hydro_to_nbody = hydro_IDs['index'].tolist()
    
    #print(hydro_to_nbody)

    ############################################################################################################################
    LOGGER.info("Matching Hydro > N-body")
    ############################################################################################################################

    starting_reordered_haloes = sorted_halo_ids[hydro_to_nbody]
    starting_matching_index = np.zeros(num_nbody_haloes)
    starting_matching_fraction = np.zeros(num_nbody_haloes)

    for count, value in enumerate(halo_mass_limit):
        starting_mask_test = starting_id_array == value
        starting_selected_ids_test = loading_dm[starting_mask_test]
        starting_selected_hydro_haloes = starting_reordered_haloes[starting_mask_test]
        starting_mode = stats.mode(starting_selected_hydro_haloes)[0]
        print(starting_mode)
        #print(starting_mode[0])
        #print(starting_mode[0][0])
        starting_matching_index[count] = starting_mode
        starting_num_matched = len(np.where(starting_selected_hydro_haloes==starting_mode)[0])
        starting_matching_fraction[count] = starting_num_matched/len(starting_selected_hydro_haloes)

    ############################################################################################################################
    LOGGER.info("Matching N-body > Hydro")
    ############################################################################################################################
    if target_suite != "Swift-EAGLE":
        hydro_IDs = pd.DataFrame(loading_dm, columns = ["IDs"])
        nbody_IDs = pd.DataFrame(fixed_hydro, columns = ["IDs"])

        hydro_IDs['index'] = hydro_IDs.index 
        hydro_IDs = hydro_IDs.set_index('IDs')
        hydro_IDs = hydro_IDs.reindex(index=nbody_IDs['IDs'])
        hydro_IDs = hydro_IDs.reset_index()

        nbody_to_hydro = hydro_IDs['index'].tolist()

    #reordering hydro halo ID array to match the order of N-body halo ID array
    reordered_haloes = starting_id_array[nbody_to_hydro]
    matching_index = np.zeros(num_hydro_haloes)
    matching_fraction = np.zeros(num_hydro_haloes)

    for count, value in enumerate(halo_mass_limit):
        starting_mask_test = sorted_halo_ids == value
        starting_selected_ids_test = fixed_hydro[starting_mask_test]
        selected_hydro_haloes = reordered_haloes[starting_mask_test]
        mode = stats.mode(selected_hydro_haloes)[0]
        matching_index[count] = mode
        num_matched = len(np.where(selected_hydro_haloes==mode)[0])
        matching_fraction[count] = num_matched/len(selected_hydro_haloes)

    matched_index_int = np.int64(matching_index)
    starting_matched_index_int = np.int64(starting_matching_index)
    double_match = np.zeros(num_hydro_haloes, dtype = int)

    for j in range(num_hydro_haloes):
        g = starting_matched_index_int[j]
        k = matched_index_int[g]
        if j == k:
            double_match[j] = 1


    #header = ['Nbody_ID', 'Hydro_ID', 'Percent_matched', 'Two-way_match']
    data = np.zeros((4, num_hydro_haloes))
    for i in range(num_hydro_haloes):
        rounded_fraction = round(starting_matching_fraction[i]*100, 2)
        data[:,i] = np.int64(halo_mass_limit[i]), np.int64(starting_matched_index_int[i]), rounded_fraction, np.int64(double_match[i])

    LOGGER.info("Writing to file")
    if two_hydro == False:
        if target_suite == "Astrid":
            hf = h5py.File(f'/home/jovyan/home/my_files/halo_matcher/{target_suite}/Nbody_{target_simulation}_{target_suite}_{target_simulation}_snap_{astrid_snap_num}.hdf5', 'w')

        else:
            hf = h5py.File(f'/home/jovyan/home/my_files/halo_matcher/{target_suite}/Nbody_{target_simulation}_{target_suite}_{target_simulation}_snap_{snap_num}.hdf5', 'w')
        hf.create_dataset('nbody_index', data = np.int64(data[0,:]))
        hf.create_dataset('hydro_index', data = np.int64(data[1,:]))
        hf.create_dataset('percent_matched', data = np.int64(data[2,:]))
        hf.create_dataset('cross_match', data = np.int64(data[3,:]))
        hf.close()
        
    if two_hydro == True:
        if target_suite == "Astrid":
            hf = h5py.File(f'/home/jovyan/home/my_files/halo_matcher/{starting_suite}/{starting_suite}_{starting_simulation}_{target_suite}_{target_simulation}_snap_{astrid_snap_num}.hdf5', 'w')

        else:
            hf = h5py.File(f'/home/jovyan/home/my_files/halo_matcher/{starting_suite}/{starting_suite}_{starting_simulation}_{target_suite}_{target_simulation}_snap_{snap_num}.hdf5', 'w')        
    
        hf.create_dataset('FirstSim_index', data = np.int64(data[0,:]))
        hf.create_dataset('SecondSim_index', data = np.int64(data[1,:]))
        hf.create_dataset('percent_matched', data = np.int64(data[2,:]))
        hf.create_dataset('cross_match', data = np.int64(data[3,:]))
        hf.close()

    LOGGER.info(f"Finished {target_simulation}")

    end = time.time()

    elapsed = end-start

    print(f'it took {elapsed} seconds to run the loop')

In [ ]:
#example of running the halo_matcher for the full CV set in SIMBA

for snap_num in ["090"]:
    suite = "SIMBA"
    for i in range(27):
        halo_matcher(target_simulation = f"CV_{i}", target_suite = suite, snap_num = snap_num)

In [ ]:
#Example of calculating virial mass values for haloes in SIMBA's LH set

radialX = 10
dr = 0.1 
boxsize = 25000
10**np.arange(-2, 1+dr, dr)

num_sim = [1000]
simfolders = ["LH"]

for snap_num in ["090"]:
    for suite in ["SIMBA"]:
        for k in range(1):
            for i in range(0, 333):
                if simfolders[k] == "1P":
                    simname = list_of_strings2[i]
                else:  
                    simname = simfolders[k] + f"_{i}"
                start = time.time()
                print(f"Starting {suite} {simname}")

                #loading hydro sim data
                LOGGER.info("loading hydro sim data")

                if suite == "Swift-EAGLE":
                    simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}/L25n256/{simfolders[k]}/{simname}/'
                    final_filename = simdir + f'snapshot_{snap_num}.hdf5'
                    z_file = np.loadtxt(simdir + "extra_files/snap_redshifts.txt")
                    #z = z_file[snap_num[-2:]]
                    #print(z)
                else:
                    simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}/L25n256/{simfolders[k]}/{simname}/'
                    final_filename = simdir + f'snapshot_{snap_num}.hdf5'
                    head = readgadget.header(final_filename)
                    z = head.redshift
                final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/{suite}/L25n256/{simfolders[k]}/{simname}/' + f'groups_{snap_num}.hdf5'
                haloes = h5py.File(final_halo_filename, 'r')

                #loading nbody sim data
                LOGGER.info("loading nbody sim data")
                #nbody_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}_DM/L25n256/{simfolders[k]}/{simname}/'
                nbody_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/L25n256/{simfolders[k]}/{simname}/'
                nbody_final_filename = nbody_simdir + f'snapshot_{snap_num}.hdf5'
                #nbody_final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/{suite}_DM/L25n256/{simfolders[k]}/{simname}/groups_{nbody_snap_num}.hdf5'
                nbody_final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/L25n256/{simfolders[k]}/{simname}/groups_{snap_num}.hdf5'
                nbody_haloes = h5py.File(nbody_final_halo_filename, 'r')
                mass = nbody_haloes['Group/Group_M_Crit200'][:]

                #loading halo data
                LOGGER.info("loading halo data")
                if suite == "Swift-EAGLE":
                    halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_{snap_num}.hdf5'
                else:
                    if suite == "Astrid":
                        halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_{snap_num}.hdf5'
                    else:
                        #halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_033.hdf5'
                        #For 1P only:
                        halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/{simfolders[k]}/Nbody_{simname}_{suite}_{simname}_snap_033.hdf5'
                gg = h5py.File(halo_matcher, 'r')
                hydro_halo_ids = gg['hydro_index'][:]
                nbody_halo_ids = gg['nbody_index'][:]
                percent_matched = gg['percent_matched'][:]
                cross_match = gg["cross_match"][:]

                cross_match_frac_mask = (percent_matched>50) & (cross_match == 1)

                crossmatched_hydro_halo_ids = hydro_halo_ids[cross_match_frac_mask]
                crossmatched_nbody_halo_ids = nbody_halo_ids[cross_match_frac_mask]

                #dm_radii = nbody_haloes['Group/Group_R_Crit200'][:][nbody_halo_ids]
                #radii = haloes['Group/Group_R_Crit200'][:][hydro_halo_ids]
                #CoMs = haloes['Group/GroupPos'][:][hydro_halo_ids]

                nbody_masses = nbody_haloes['Group/Group_M_Crit200'][:]
                selected_nbody_masses = nbody_masses[crossmatched_nbody_halo_ids]
                nbody_logmasses = np.log10(selected_nbody_masses*1e10)

                nbody_massindex = crossmatched_nbody_halo_ids
                massindex = crossmatched_hydro_halo_ids

                num_haloes = len(massindex)
                num_rows = np.max(np.maximum(massindex, nbody_massindex)) + 1

                #loading particle data
                LOGGER.info("loading particle data")
                with h5py.File(final_filename, "r") as handle:
                    dark_matter_final_coordinates = np.array(handle["PartType1/Coordinates"][:])
                    dark_matter_final_ids = np.array(handle["PartType1/ParticleIDs"][:])
                    gas_final_coordinates = np.array(handle["PartType0/Coordinates"][:])
                    gas_final_ids = np.array(handle["PartType0/ParticleIDs"][:])
                    gas_final_masses = np.array(handle["PartType0/Masses"][:])*1e10
                    star_final_coordinates = np.array(handle["PartType4/Coordinates"][:])
                    star_final_ids = np.array(handle["PartType4/ParticleIDs"][:])
                    star_final_masses = np.array(handle["PartType4/Masses"][:])*1e10
                    bh_final_coordinates = np.array(handle["PartType5/Coordinates"][:])
                    bh_final_ids = np.array(handle["PartType5/ParticleIDs"][:])

                    if suite == "Swift-EAGLE":
                        dark_matter_final_masses = np.array(handle["PartType1/Masses"][:])*1e10 * 0.6711
                        gas_final_masses = np.array(handle["PartType0/Masses"][:])*1e10 * 0.6711
                        star_final_masses = np.array(handle["PartType4/Masses"][:])*1e10 * 0.6711
                        bh_final_masses = np.array(handle["PartType5/SubgridMasses"][:])*1e10 * 0.6711

                        dark_matter_final_coordinates = dark_matter_final_coordinates * 0.6711 * 1000
                        gas_final_coordinates = gas_final_coordinates * 0.6711 * 1000
                        star_final_coordinates = star_final_coordinates * 0.6711 * 1000
                        bh_final_coordinates = bh_final_coordinates * 0.6711 * 1000

                    else:
                        bh_final_masses = np.array(handle["PartType5/Masses"][:])*1e10 

                    if suite == "IllustrisTNG":
                        dmmass = handle["Header"].attrs["MassTable"][1] *1e10
                        dark_matter_final_masses = np.zeros(len(dark_matter_final_ids)) + dmmass
                    if suite == "SIMBA" or suite == "Astrid":
                        dark_matter_final_masses = np.array(handle["PartType1/Masses"][:])*1e10

                    ind_check0 = np.where(dark_matter_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        dark_matter_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(dark_matter_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        dark_matter_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(dark_matter_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        dark_matter_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                    ind_check0 = np.where(gas_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        gas_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(gas_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        gas_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(gas_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        gas_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                    ind_check0 = np.where(star_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        star_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(star_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        star_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(star_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        star_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                    LOGGER.info("Building hydro trees")

                    gas_tree = cKDTree(gas_final_coordinates, boxsize=boxsize)
                    star_tree = cKDTree(star_final_coordinates, boxsize=boxsize)
                    bh_tree = cKDTree(bh_final_coordinates, boxsize=boxsize)
                    dark_matter_tree = cKDTree(dark_matter_final_coordinates, boxsize=boxsize)

                #loading n-body data
                LOGGER.info("loading n-body data")
                with h5py.File(nbody_final_filename, "r") as dm_handle:
                    nbody_dark_matter_final_coordinates = np.array(dm_handle["PartType1/Coordinates"][:])
                    nbody_dark_matter_final_ids = np.array(dm_handle["PartType1/ParticleIDs"][:])
                    dmmass = dm_handle["Header"].attrs["MassTable"][1] *1e10
                    nbody_dark_matter_final_masses = np.zeros(len(nbody_dark_matter_final_ids)) + dmmass 

                    ind_check0 = np.where(nbody_dark_matter_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        nbody_dark_matter_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(nbody_dark_matter_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        nbody_dark_matter_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(nbody_dark_matter_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        nbody_dark_matter_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate")


                    LOGGER.info("Building n-body trees")

                    nbody_dark_matter_tree = cKDTree(nbody_dark_matter_final_coordinates, boxsize=boxsize)

                root = "/home/jovyan/home/my_radial_profiles"
                full_root = root + f'/{suite}/{simname[:2]}/{simname}/'
                dm_full_root = root + f'/SIMBA_DM/{simname}/'

                total_virial_mass_array = np.empty(num_rows)
                gas_virial_mass_array = np.empty(num_rows)
                dm_virial_mass_array = np.empty(num_rows)
                stars_virial_mass_array = np.empty(num_rows)
                bh_virial_mass_array = np.empty(num_rows)
                Rvir_array = np.empty(num_rows)
                nbody_virial_mass_array = np.empty(num_rows)
                nbody_Rvir_array = np.empty(num_rows)

                for j in range(num_haloes):
                    halo_index = int(massindex[j])
                    nbody_halo_index = nbody_massindex[j]
                    (total_virial_mass_array[halo_index], 
                    dm_virial_mass_array[halo_index], 
                    gas_virial_mass_array[halo_index], 
                    stars_virial_mass_array[halo_index], 
                    bh_virial_mass_array[halo_index],
                    Rvir_array[halo_index]) = virial_mass_calculator(halo_index)

                    (nbody_virial_mass_array[nbody_halo_index], 
                     nbody_Rvir_array[nbody_halo_index]) = nbody_virial_mass_calculator(nbody_halo_index)

                #print(total_virial_mass_array)

                halo_filepath = f'/home/jovyan/home/halo_data2/{suite}/{simname[:2]}/halo_data2_{suite}_{simname}_snap_{snap_num}.hdf5'
                halo_file = h5py.File(halo_filepath, 'a')

                Mvir = halo_file.create_group('virial_masses')

                Mvir_hydro = Mvir.create_group('hydro')
                Mvir_Nbody = Mvir.create_group('Nbody')

                Mvir_total = Mvir_hydro.create_dataset('total_matter', data = total_virial_mass_array)
                Mvir_dark_matter = Mvir_hydro.create_dataset('dark_matter', data = dm_virial_mass_array)
                Mvir_gas = Mvir_hydro.create_dataset('gas', data = gas_virial_mass_array)
                Mvir_stars = Mvir_hydro.create_dataset('stars', data = stars_virial_mass_array)
                Mvir_black_holes = Mvir_hydro.create_dataset('black_holes', data = bh_virial_mass_array)

                Mvir_nbody_total = Mvir_Nbody.create_dataset('total_matter', data = nbody_virial_mass_array)

                Rvir = halo_file.create_group('virial_radii')

                Rvir_hydro = Rvir.create_dataset('hydro_radii', data = Rvir_array)
                Rvir_nbody = Rvir.create_dataset('Nbody_radii', data = nbody_Rvir_array)
                
                halo_file.close()

In [ ]:
#Example of calculating density profiles for haloes in SIMBA's LH set

radialX = 10
dr = 0.1 
boxsize = 25000

z = 0.00

num_bins = len(10**np.arange(-2, 1+dr, dr))
radial_bin_array_for_file = 10**np.arange(-2, 1+dr, dr)

num_sim = [1000]
simfolders = ["LH"]
for snap_num in ["090"]:
    nbody_snap_num = snap_num
    for suite in ["SIMBA"]:
    #for suite in ["SIMBA"]:
        #if suite == "Eagle":
        for k in range(1):
            for i in range(0, 333):
            #for i in range(2,3):
                if simfolders[k] == "1P":
                    simname = list_of_strings2[i]
                else:  
                    simname = simfolders[k] + f"_{i}"
                start = time.time()
                print(f"Starting {suite} {simname}")

                #loading hydro sim data
                LOGGER.info("loading hydro sim data")

                if suite == "Swift-EAGLE":
                    simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}/L25n256/{simfolders[k]}/{simname}/'
                    final_filename = simdir + f'snapshot_{snap_num}.hdf5'
                    #z_file = np.loadtxt(simdir + "extra_files/snap_redshifts.txt")
                    #z = z_file[snap_num[-2:]]
                    #print(z)
                else:
                    simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}/{simfolders[k]}/{simname}/'
                    final_filename = simdir + f'snapshot_{snap_num}.hdf5'
                    head = readgadget.header(final_filename)
                    z = head.redshift
                final_halo_filename = simdir + f'groups_{snap_num}.hdf5'
                haloes = h5py.File(final_halo_filename, 'r')

                #loading nbody sim data
                LOGGER.info("loading nbody sim data")
                nbody_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/{simfolders[k]}/{simname}/'
                nbody_final_filename = nbody_simdir + f'snapshot_{nbody_snap_num}.hdf5'
                nbody_final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/{simfolders[k]}/{simname}/groups_{nbody_snap_num}.hdf5'
                nbody_haloes = h5py.File(nbody_final_halo_filename, 'r')
                mass = nbody_haloes['Group/Group_M_Crit200'][:]

                #loading halo data
                LOGGER.info("loading halo data")
                if suite == "Swift-EAGLE":
                    halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_Eagle_{simname}_snap_{snap_num}.hdf5'
                else:
                    if suite == "Astrid":
                        halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_{snap_num}.hdf5'
                    else:
                        #halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_033.hdf5'
                        #For 1P only:
                        halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/{simfolders[k]}/Nbody_{simname}_{suite}_{simname}_snap_033.hdf5'
                gg = h5py.File(halo_matcher, 'r')
                hydro_halo_ids = gg['hydro_index'][:]
                nbody_halo_ids = gg['nbody_index'][:]
                percent_matched = gg['percent_matched'][:]
                cross_match = gg["cross_match"][:]

                cross_match_frac_mask = (percent_matched>50) & (cross_match == 1)

                crossmatched_hydro_halo_ids = hydro_halo_ids[cross_match_frac_mask]
                crossmatched_nbody_halo_ids = nbody_halo_ids[cross_match_frac_mask]

                #dm_radii = nbody_haloes['Group/Group_R_Crit200'][:][nbody_halo_ids]
                #radii = haloes['Group/Group_R_Crit200'][:][hydro_halo_ids]
                #CoMs = haloes['Group/GroupPos'][:][hydro_halo_ids]

                nbody_masses = nbody_haloes['Group/Group_M_Crit200'][:]
                selected_nbody_masses = nbody_masses[crossmatched_nbody_halo_ids]
                nbody_logmasses = np.log10(selected_nbody_masses*1e10)

                nbody_mask = (nbody_logmasses > 11)
                nbody_massindex = crossmatched_nbody_halo_ids[nbody_mask]
                massindex = crossmatched_hydro_halo_ids[nbody_mask]

                num_haloes = len(massindex)

                num_rows = np.max(np.maximum(massindex, nbody_massindex)) + 1

                #loading particle data
                LOGGER.info("loading particle data")
                with h5py.File(final_filename, "r") as handle:
                    dark_matter_final_coordinates = np.array(handle["PartType1/Coordinates"][:])
                    dark_matter_final_ids = np.array(handle["PartType1/ParticleIDs"][:])
                    gas_final_coordinates = np.array(handle["PartType0/Coordinates"][:])
                    gas_final_ids = np.array(handle["PartType0/ParticleIDs"][:])
                    gas_final_masses = np.array(handle["PartType0/Masses"][:])*1e10
                    star_final_coordinates = np.array(handle["PartType4/Coordinates"][:])
                    star_final_ids = np.array(handle["PartType4/ParticleIDs"][:])
                    star_final_masses = np.array(handle["PartType4/Masses"][:])*1e10
                    bh_final_coordinates = np.array(handle["PartType5/Coordinates"][:])
                    bh_final_ids = np.array(handle["PartType5/ParticleIDs"][:])

                    if suite == "Swift-EAGLE":
                        dark_matter_final_masses = np.array(handle["PartType1/Masses"][:])*1e10 * 0.6711
                        gas_final_masses = np.array(handle["PartType0/Masses"][:])*1e10 * 0.6711
                        star_final_masses = np.array(handle["PartType4/Masses"][:])*1e10 * 0.6711
                        bh_final_masses = np.array(handle["PartType5/SubgridMasses"][:])*1e10 * 0.6711

                        dark_matter_final_coordinates = dark_matter_final_coordinates * 0.6711 * 1000
                        gas_final_coordinates = gas_final_coordinates * 0.6711 * 1000
                        star_final_coordinates = star_final_coordinates * 0.6711 * 1000
                        bh_final_coordinates = bh_final_coordinates * 0.6711 * 1000

                    else:
                        bh_final_masses = np.array(handle["PartType5/Masses"][:])*1e10 

                    if suite == "IllustrisTNG":
                        dmmass = handle["Header"].attrs["MassTable"][1] *1e10
                        dark_matter_final_masses = np.zeros(len(dark_matter_final_ids)) + dmmass
                    if suite == "SIMBA" or suite == "Astrid":
                        dark_matter_final_masses = np.array(handle["PartType1/Masses"][:])*1e10

                    #print(gas_final_masses)
                    #print(dark_matter_final_masses)
                    #print(star_final_masses)
                    #print(bh_final_masses)

                    ind_check0 = np.where(dark_matter_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        dark_matter_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(dark_matter_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        dark_matter_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(dark_matter_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        dark_matter_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                    ind_check0 = np.where(gas_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        gas_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(gas_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        gas_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(gas_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        gas_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                    ind_check0 = np.where(star_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        star_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(star_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        star_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(star_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        star_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                    LOGGER.info("Building hydro trees")

                    gas_tree = cKDTree(gas_final_coordinates, boxsize=boxsize)
                    star_tree = cKDTree(star_final_coordinates, boxsize=boxsize)
                    bh_tree = cKDTree(bh_final_coordinates, boxsize=boxsize)
                    dark_matter_tree = cKDTree(dark_matter_final_coordinates, boxsize=boxsize)

                #loading n-body data
                LOGGER.info("loading n-body data")
                with h5py.File(nbody_final_filename, "r") as dm_handle:
                    nbody_dark_matter_final_coordinates = np.array(dm_handle["PartType1/Coordinates"][:])
                    nbody_dark_matter_final_ids = np.array(dm_handle["PartType1/ParticleIDs"][:])
                    dmmass = dm_handle["Header"].attrs["MassTable"][1] *1e10
                    nbody_dark_matter_final_masses = np.zeros(len(nbody_dark_matter_final_ids)) + dmmass 

                    ind_check0 = np.where(nbody_dark_matter_final_coordinates[:,0] >= boxsize)[0]
                    if ind_check0.any() == True:
                        nbody_dark_matter_final_coordinates[ind_check0, 0] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                    ind_check1 = np.where(nbody_dark_matter_final_coordinates[:,1] >= boxsize)[0]
                    if ind_check1.any() == True:
                        nbody_dark_matter_final_coordinates[ind_check1, 1] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                    ind_check2 = np.where(nbody_dark_matter_final_coordinates[:,2] >= boxsize)[0]
                    if ind_check2.any() == True:
                        nbody_dark_matter_final_coordinates[ind_check2, 2] = boxsize - 0.001
                        LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate")


                    LOGGER.info("Building n-body trees")

                    nbody_dark_matter_tree = cKDTree(nbody_dark_matter_final_coordinates, boxsize=boxsize)

                root = "/home/jovyan/home/my_radial_profiles"
                full_root = root + f'/{suite}/{simname[:2]}/{simname}/'
                dm_full_root = root + f'/SIMBA_DM/{simname}/'

                #print(nbody_massindex)

                total_hist = np.empty((num_rows, num_bins))
                gas_hist = np.empty((num_rows, num_bins))
                dm_hist = np.empty((num_rows, num_bins))
                stars_hist = np.empty((num_rows, num_bins))
                nbody_hist = np.empty((num_rows, num_bins))

                for i in range(len(nbody_massindex)):
                #for i in range(2):
                    #calculating n-body profile
                    nbody_index = nbody_massindex[i]
                    #LOGGER.info(f"Calculating profile for n-body halo {nbody_index}")
                    (nbody_dark_matter_in_halo_masses, nbody_dark_matter_radial_value, 
                    nbody_radius_array, nbody_radius) = nbody_profile_data_selector(nbody_index)
                    bins = len(nbody_radius_array)
                    (nbody_total_density_array) = nbody_profile_calculator(bins, nbody_dark_matter_in_halo_masses, 
                                                                     nbody_dark_matter_radial_value, nbody_radius_array, nbody_radius)

                    #print("driver nbody_radius_array", nbody_radius_array)
                    dm_total_path = dm_full_root + f'total_density_halo_{nbody_index}_z=%.2f' % z

                    #np.savetxt(dm_total_path, np.transpose([nbody_radius_array, nbody_total_density_array]), delimiter='\t')


                    halo_index = massindex[i]
                    #LOGGER.info(f"Calculating profile for hydro halo {halo_index}")
                    (dark_matter_in_halo_masses, dark_matter_radial_value, 
                    gas_in_halo_masses, gas_radial_value, 
                    star_in_halo_masses, star_radial_value, bh_in_halo_masses, 
                    bh_radial_value, radius_array, radius) = profile_data_selector_radial_fix(halo_index, nbody_index)
                    #print("driver nbody_radius_array2", nbody_radius_array)
                    (total_density_array, dm_density_array, 
                    gas_density_array, star_density_array, 
                    bh_density_array) = total_profile_calculator_radial_fix(bins, dark_matter_in_halo_masses, gas_in_halo_masses, 
                                                                            star_in_halo_masses, bh_in_halo_masses, dark_matter_radial_value, 
                                                                            gas_radial_value, star_radial_value, bh_radial_value, nbody_radius_array, 
                                                                            nbody_radius)


                    total_hist[halo_index,:] = total_density_array
                    gas_hist[halo_index,:] = gas_density_array
                    dm_hist[halo_index,:] = dm_density_array
                    stars_hist[halo_index,:] = bh_density_array

                    nbody_hist[nbody_index,:] = nbody_total_density_array

                halo_filepath = f'/home/jovyan/home/halo_data2/{suite}/{simname[:2]}/halo_data2_{suite}_{simname}_snap_{snap_num}.hdf5'
                halo_file = h5py.File(halo_filepath, 'a')
                    
                #temp_total_hist = halo_file['hydro_matched_density_profiles/total_matter']
                #temp_dm_hist = halo_file['hydro_matched_density_profiles/dark_matter']
                #temp_gas_hist = halo_file['hydro_matched_density_profiles/gas']
                #temp_stars_hist = halo_file['hydro_matched_density_profiles/stars']
                    
                #temp_nbody_hist = halo_file['Nbody_density_profiles']
                    
                #temp_radial_bin_array_for_file = halo_file['bins']
                    
                #temp_crossmatched_hydro_halo_ids = halo_file['halo_IDs/hydro_IDs']
                #temp_crossmatched_nbody_halo_ids = halo_file['halo_IDs/Nbody_IDs']
                    
                #temp_total_hist[...] = total_hist
                #temp_dm_hist[...] = dm_hist
                #temp_gas_hist[...] = gas_hist
                #temp_stars_hist[...] = stars_hist
                    
                #temp_nbody_hist[...] = nbody_hist
                    
                #temp_radial_bin_array_for_file[...] = radial_bin_array_for_file
                    
                #temp_crossmatched_hydro_halo_ids[...] = crossmatched_hydro_halo_ids
                #temp_crossmatched_nbody_halo_ids[...] = crossmatched_nbody_halo_ids

                NR_den_pro = halo_file.create_group('hydro_matched_density_profiles')

                NRDP_total = NR_den_pro.create_dataset('total_matter', data = total_hist)
                NRDP_dm = NR_den_pro.create_dataset('dark_matter', data = dm_hist)
                NRDP_gas = NR_den_pro.create_dataset('gas', data = gas_hist)
                NRDP_stars = NR_den_pro.create_dataset('stars', data = stars_hist)

                nbody_den_pro = halo_file.create_dataset('Nbody_density_profiles', data = nbody_hist)

                rad_arr = halo_file.create_dataset('bins', data = radial_bin_array_for_file)

                IDs = halo_file.create_group('halo_IDs')
                hydro_ID_for_file = IDs.create_dataset('hydro_IDs', data = crossmatched_hydro_halo_ids)
                Nbody_ID_for_file = IDs.create_dataset('Nbody_IDs', data = crossmatched_nbody_halo_ids)
                    
                #print(total_hist[:10,:])
                #print(halo_file['hydro_matched_density_profiles/total_matter'][:10,:])
                
                halo_file.close()

In [ ]:
#Example of calculating mass profiles for haloes in SIMBA's LH set

snap_num = "090"
nbody_snap_num = "090"
radialX = 10
dr = 0.1 
boxsize = 25000
num_bins = len(10**np.arange(-2, 1+dr, dr))

num_sim = [1000]
simfolders = ["LH"]

#for suite in ["Astrid"]:
for suite in ["SIMBA"]:
#for suite in ["Swift-EAGLE"]:
    #if suite == "Eagle":
    snap_num = "090"
    for k in range(1):
        for i in range(num_sim[k]):
            if simfolders[k] == "1P":
                simname = list_of_strings2[i]
            else:  
                simname = simfolders[k] + f"_{i}"
            start = time.time()
            print(f"Starting {suite} {simname}")

            #loading hydro sim data
            LOGGER.info("loading hydro sim data")
            
            if suite == "Swift-EAGLE":
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}/L25n256/{simfolders[k]}/{simname}/'
                final_filename = simdir + f'snapshot_{snap_num}.hdf5'
                z_file = np.loadtxt(simdir + "extra_files/snap_redshifts.txt")
                z = z_file[90]
                #print(z)
            else:
                simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}/{simfolders[k]}/{simname}/'
                final_filename = simdir + f'snapshot_{snap_num}.hdf5'
                head = readgadget.header(final_filename)
                z = head.redshift
            final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/{suite}/L25n256/{simfolders[k]}/{simname}/' + f'groups_{snap_num}.hdf5'
            haloes = h5py.File(final_halo_filename, 'r')

            #loading nbody sim data
            LOGGER.info("loading nbody sim data")
            nbody_simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/SIMBA_DM/{simfolders[k]}/{simname}/'
            nbody_final_filename = nbody_simdir + f'snapshot_{nbody_snap_num}.hdf5'
            nbody_final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/SIMBA_DM/{simfolders[k]}/{simname}/groups_{nbody_snap_num}.hdf5'
            nbody_haloes = h5py.File(nbody_final_halo_filename, 'r')
            mass = nbody_haloes['Group/Group_M_Crit200'][:]
            
            num_particles_in_hydro = haloes['Group/GroupLen'][:]
            num_mask = (num_particles_in_hydro > 100)
            halo_mass_limit = np.where(num_mask == True)[0]
            num_total_haloes = len(halo_mass_limit)

            #loading halo data
            LOGGER.info("loading halo data")
            if suite == "Swift-EAGLE":
                halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_Eagle_{simname}_snap_{snap_num}.hdf5'
            else:
                if suite == "Astrid":
                    halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_{snap_num}.hdf5'
                else:
                    #halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_033.hdf5'
                    #For 1P only:
                    halo_matcher = f'/home/jovyan/home/my_files/halo_matcher/{suite}/Nbody_{simname}_{suite}_{simname}_snap_{snap_num}.hdf5'
            gg = h5py.File(halo_matcher, 'r')
            hydro_halo_ids = gg['hydro_index'][:]
            nbody_halo_ids = gg['nbody_index'][:]
            percent_matched = gg['percent_matched'][:]
            cross_match = gg["cross_match"][:]

            cross_match_frac_mask = (percent_matched>50) & (cross_match == 1)

            crossmatched_hydro_halo_ids = hydro_halo_ids[cross_match_frac_mask]
            crossmatched_nbody_halo_ids = nbody_halo_ids[cross_match_frac_mask]

            #dm_radii = nbody_haloes['Group/Group_R_Crit200'][:][nbody_halo_ids]
            #radii = haloes['Group/Group_R_Crit200'][:][hydro_halo_ids]
            #CoMs = haloes['Group/GroupPos'][:][hydro_halo_ids]

            nbody_masses = nbody_haloes['Group/Group_M_Crit200'][:]
            selected_nbody_masses = nbody_masses[crossmatched_nbody_halo_ids]
            nbody_logmasses = np.log10(selected_nbody_masses*1e10)

            nbody_massindex = crossmatched_nbody_halo_ids
            massindex = crossmatched_hydro_halo_ids

            num_haloes = len(massindex)
            num_rows = np.max(np.maximum(massindex, nbody_massindex)) + 1
            
            print(num_rows)

            #loading particle data
            LOGGER.info("loading particle data")
            with h5py.File(final_filename, "r") as handle:
                dark_matter_final_coordinates = np.array(handle["PartType1/Coordinates"][:])
                dark_matter_final_ids = np.array(handle["PartType1/ParticleIDs"][:])
                gas_final_coordinates = np.array(handle["PartType0/Coordinates"][:])
                gas_final_ids = np.array(handle["PartType0/ParticleIDs"][:])
                gas_final_masses = np.array(handle["PartType0/Masses"][:])*1e10
                star_final_coordinates = np.array(handle["PartType4/Coordinates"][:])
                star_final_ids = np.array(handle["PartType4/ParticleIDs"][:])
                star_final_masses = np.array(handle["PartType4/Masses"][:])*1e10
                bh_final_coordinates = np.array(handle["PartType5/Coordinates"][:])
                bh_final_ids = np.array(handle["PartType5/ParticleIDs"][:])
                
                if suite == "Swift-EAGLE":
                    dark_matter_final_masses = np.array(handle["PartType1/Masses"][:])*1e10 * 0.6711
                    gas_final_masses = np.array(handle["PartType0/Masses"][:])*1e10 * 0.6711
                    star_final_masses = np.array(handle["PartType4/Masses"][:])*1e10 * 0.6711
                    bh_final_masses = np.array(handle["PartType5/SubgridMasses"][:])*1e10 * 0.6711
                    
                    dark_matter_final_coordinates = dark_matter_final_coordinates * 0.6711 * 1000
                    gas_final_coordinates = gas_final_coordinates * 0.6711 * 1000
                    star_final_coordinates = star_final_coordinates * 0.6711 * 1000
                    bh_final_coordinates = bh_final_coordinates * 0.6711 * 1000
                    
                else:
                    bh_final_masses = np.array(handle["PartType5/Masses"][:])*1e10 
                
                if suite == "IllustrisTNG":
                    dmmass = handle["Header"].attrs["MassTable"][1] *1e10
                    dark_matter_final_masses = np.zeros(len(dark_matter_final_ids)) + dmmass
                if suite == "SIMBA" or suite == "Astrid":
                    dark_matter_final_masses = np.array(handle["PartType1/Masses"][:])*1e10
                
                #print(gas_final_masses)
                #print(dark_matter_final_masses)
                #print(star_final_masses)
                #print(bh_final_masses)

                ind_check0 = np.where(dark_matter_final_coordinates[:,0] >= boxsize)[0]
                if ind_check0.any() == True:
                    dark_matter_final_coordinates[ind_check0, 0] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                ind_check1 = np.where(dark_matter_final_coordinates[:,1] >= boxsize)[0]
                if ind_check1.any() == True:
                    dark_matter_final_coordinates[ind_check1, 1] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                ind_check2 = np.where(dark_matter_final_coordinates[:,2] >= boxsize)[0]
                if ind_check2.any() == True:
                    dark_matter_final_coordinates[ind_check2, 2] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                ind_check0 = np.where(gas_final_coordinates[:,0] >= boxsize)[0]
                if ind_check0.any() == True:
                    gas_final_coordinates[ind_check0, 0] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                ind_check1 = np.where(gas_final_coordinates[:,1] >= boxsize)[0]
                if ind_check1.any() == True:
                    gas_final_coordinates[ind_check1, 1] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                ind_check2 = np.where(gas_final_coordinates[:,2] >= boxsize)[0]
                if ind_check2.any() == True:
                    gas_final_coordinates[ind_check2, 2] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                ind_check0 = np.where(star_final_coordinates[:,0] >= boxsize)[0]
                if ind_check0.any() == True:
                    star_final_coordinates[ind_check0, 0] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                ind_check1 = np.where(star_final_coordinates[:,1] >= boxsize)[0]
                if ind_check1.any() == True:
                    star_final_coordinates[ind_check1, 1] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                ind_check2 = np.where(star_final_coordinates[:,2] >= boxsize)[0]
                if ind_check2.any() == True:
                    star_final_coordinates[ind_check2, 2] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate") 

                LOGGER.info("Building hydro trees")

                gas_tree = cKDTree(gas_final_coordinates, boxsize=boxsize)
                star_tree = cKDTree(star_final_coordinates, boxsize=boxsize)
                bh_tree = cKDTree(bh_final_coordinates, boxsize=boxsize)
                dark_matter_tree = cKDTree(dark_matter_final_coordinates, boxsize=boxsize)

            #loading n-body data
            LOGGER.info("loading n-body data")
            with h5py.File(nbody_final_filename, "r") as dm_handle:
                nbody_dark_matter_final_coordinates = np.array(dm_handle["PartType1/Coordinates"][:])
                nbody_dark_matter_final_ids = np.array(dm_handle["PartType1/ParticleIDs"][:])
                dmmass = dm_handle["Header"].attrs["MassTable"][1] *1e10
                nbody_dark_matter_final_masses = np.zeros(len(nbody_dark_matter_final_ids)) + dmmass 

                ind_check0 = np.where(nbody_dark_matter_final_coordinates[:,0] >= boxsize)[0]
                if ind_check0.any() == True:
                    nbody_dark_matter_final_coordinates[ind_check0, 0] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the X coordinate")
                ind_check1 = np.where(nbody_dark_matter_final_coordinates[:,1] >= boxsize)[0]
                if ind_check1.any() == True:
                    nbody_dark_matter_final_coordinates[ind_check1, 1] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Y coordinate")
                ind_check2 = np.where(nbody_dark_matter_final_coordinates[:,2] >= boxsize)[0]
                if ind_check2.any() == True:
                    nbody_dark_matter_final_coordinates[ind_check2, 2] = boxsize - 0.001
                    LOGGER.info(f"Fixed index {ind_check0} of the Z coordinate")


                LOGGER.info("Building n-body trees")

                nbody_dark_matter_tree = cKDTree(nbody_dark_matter_final_coordinates, boxsize=boxsize)
                
            total_int_mass_array = np.empty((num_rows, num_bins))
            gas_int_mass_array = np.empty((num_rows, num_bins))
            dm_int_mass_array = np.empty((num_rows, num_bins))
            stars_int_mass_array = np.empty((num_rows, num_bins))
            bh_int_mass_array = np.empty((num_rows, num_bins))
            nbody_int_mass_array = np.empty((num_rows, num_bins))
            
            for i in range(len(nbody_massindex)):
                #calculating n-body profile
                nbody_index = nbody_massindex[i]
                (nbody_dark_matter_in_halo_masses, nbody_dark_matter_radial_value, 
                nbody_radius_array, nbody_radius) = nbody_profile_data_selector(nbody_index)
                bins = len(nbody_radius_array)
                (nbody_total_int_array) = nbody_mass_profile_calculator(bins, nbody_dark_matter_in_halo_masses, 
                                                                 nbody_dark_matter_radial_value, nbody_radius_array, nbody_radius)


                halo_index = massindex[i]
                (dark_matter_in_halo_masses, dark_matter_radial_value, 
                gas_in_halo_masses, gas_radial_value, 
                star_in_halo_masses, star_radial_value, bh_in_halo_masses, 
                bh_radial_value, radius_array, radius) = profile_data_selector_radial_fix(halo_index, nbody_index)
                (total_int_array, dm_int_array, 
                gas_int_array, star_int_array, 
                bh_int_array) = total_mass_profile_calculator_radial_fix(bins, dark_matter_in_halo_masses, gas_in_halo_masses, 
                                                                        star_in_halo_masses, bh_in_halo_masses, dark_matter_radial_value, 
                                                                        gas_radial_value, star_radial_value, bh_radial_value, nbody_radius_array, 
                                                                        nbody_radius)
                
                total_int_mass_array[halo_index,:] = total_int_array
                gas_int_mass_array[halo_index,:] = gas_int_array
                dm_int_mass_array[halo_index,:] = dm_int_array
                stars_int_mass_array[halo_index,:] = star_int_array
                nbody_int_mass_array[nbody_index,:] = nbody_total_int_array
                
            halo_filepath = f'/home/jovyan/home/my_files/halo_data2/{suite}/{simname[:2]}/halo_data2_{suite}_{simname}_snap_{snap_num}.hdf5'
            halo_file = h5py.File(halo_filepath, 'a')
            
            NR_mass_pro = halo_file.create_group('hydro_mass_profiles')

            NRMP_total = NR_mass_pro.create_dataset('total_matter', data = total_int_mass_array)
            NRMP_dm = NR_mass_pro.create_dataset('dark_matter', data = dm_int_mass_array)
            NRMP_gas = NR_mass_pro.create_dataset('gas', data = gas_int_mass_array)
            NRMP_stars = NR_mass_pro.create_dataset('stars', data = stars_int_mass_array)
            
            nbody_mass_pro = halo_file.create_dataset('Nbody_mass_profiles', data = nbody_int_mass_array)
            
            halo_file.close()